# Propósito da análise

Com essa base de dados espero obter informações sobre:

- Ataques de tubarões nos últimos 50 anos
- Realizando qual atividade as pessoas sofreram mais ataques
- Desses ataques, quais foram fatais
- Dos que não foram fatais, quais deixaram sequelas
- Qual país tem mais ataques de tubarão


## importando bibliotecas e a base de dados 


In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv("../../shark_project/shark_attack/attacks.csv", encoding='latin1')

## Explorando

In [3]:
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(25723, 24)

In [5]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [7]:
df.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [8]:
# O que as últimas colunas contem?
df[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']].head()

,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [9]:
# Essas colunas não contém informações relevantes para a minha análise e podem ser removidas

In [10]:
# Existem linhas duplicadas?
print('Linhas duplicadas:' )
print(df.duplicated().sum())

Linhas duplicadas:
19411


# Limpeza

## Remover duplicatas

In [11]:
shark_attack = df.drop_duplicates()
print('Número de duplicatas removidas: ', len(df) - len(shark_attack))

Número de duplicatas removidas:  19411


In [12]:
shark_attack.shape

(6312, 24)

## Remover colunas irrelevantes

In [13]:
shark_attack = shark_attack.drop(['Case Number', 'Date','Location','Name','Sex ','Age','Time', 'Species ','Investigator or Source', 'pdf', 'href formula', 'href','Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23'], axis =1)

In [14]:
shark_attack.columns

Index(['Year', 'Type', 'Country', 'Area', 'Activity', 'Injury', 'Fatal (Y/N)'], dtype='object')

In [15]:
shark_attack.loc[:, "Country":"Activity"] = shark_attack.loc[:, "Country":"Activity"].transform(lambda series: series.str.strip().str.title())

In [16]:
shark_attack.head(10)

,Year,Type,Country,Area,Activity,Injury,Fatal (Y/N)
0,2018.0,Boating,Usa,California,Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,2018.0,Unprovoked,Usa,Georgia,Standing,Minor injury to left thigh,N
2,2018.0,Invalid,Usa,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,N
3,2018.0,Unprovoked,Australia,New South Wales,Surfing,Minor injury to lower leg,N
4,2018.0,Provoked,Mexico,Colima,Free Diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N
5,2018.0,Unprovoked,Australia,New South Wales,Kite Surfing,"No injury, board bitten",N
6,2018.0,Unprovoked,Brazil,Pernambuco,Swimming,FATAL,Y
7,2018.0,Unprovoked,Usa,Florida,Fishing,Minor injury to foot. PROVOKED INCIDENT,N
8,2018.0,Unprovoked,Usa,Florida,Walking,Lower left leg bitten,N
9,2018.0,Unprovoked,Usa,Florida,Standing,Minor injury to foot,N


In [17]:
shark_attack = shark_attack.rename(columns={'Fatal (Y/N)': 'Fatal'})

In [18]:
shark_attack.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6312 entries, 0 to 25722
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      6300 non-null   float64
 1   Type      6298 non-null   object 
 2   Country   6252 non-null   object 
 3   Area      5847 non-null   object 
 4   Activity  5758 non-null   object 
 5   Injury    6274 non-null   object 
 6   Fatal     5763 non-null   object 
dtypes: float64(1), object(6)
memory usage: 394.5+ KB


In [19]:
shark_attack.isnull().sum()

Year         12
Type         14
Country      60
Area        465
Activity    554
Injury       38
Fatal       549
dtype: int64

## Coluna Year

In [20]:
shark_attack['Year'].unique()

array([2018., 2017.,   nan, 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [21]:
shark_attack.Year.isnull().sum()

12

In [22]:
shark_attack = shark_attack.dropna(subset=['Year'])

In [23]:
shark_attack.Year.isnull().sum()

0

In [24]:
shark_attack.Year = shark_attack.Year.astype(np.int64)
shark_attack['Year'].unique()

array([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008,
       2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997,
       1996, 1995, 1984, 1994, 1993, 1992, 1991, 1990, 1989, 1969, 1988,
       1987, 1986, 1985, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976,
       1975, 1974, 1973, 1972, 1971, 1970, 1968, 1967, 1966, 1965, 1964,
       1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953,
       1952, 1951, 1950, 1949, 1948, 1848, 1947, 1946, 1945, 1944, 1943,
       1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934, 1933, 1932,
       1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923, 1922, 1921,
       1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911, 1910,
       1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900, 1899,
       1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891, 1890, 1889, 1888,
       1887, 1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878, 1877,
       1876, 1875, 1874, 1873, 1872, 1871, 1870, 18

In [25]:
shark_attack = shark_attack[shark_attack['Year'] > 1967] 

shark_attack

,Year,Type,Country,Area,Activity,Injury,Fatal
0,2018,Boating,Usa,California,Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,2018,Unprovoked,Usa,Georgia,Standing,Minor injury to left thigh,N
2,2018,Invalid,Usa,Hawaii,Surfing,Injury to left lower leg from surfboard skeg,N
3,2018,Unprovoked,Australia,New South Wales,Surfing,Minor injury to lower leg,N
4,2018,Provoked,Mexico,Colima,Free Diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...
3488,1968,Unprovoked,Papua New Guinea,Morobe Province,Swimming,FATAL,Y
3489,1968,Unprovoked,Costa Rica,Limón Province,Swimming After His Canoe Capsized,FATAL,Y
3490,1968,Invalid,Usa,Florida,Diving,"Recovered, said to involve a 1m nurse shark",NaN
3491,1968,Invalid,Usa,Florida,Surfing,NaN,NaN


## Coluna Activity

In [26]:
shark_attack["Activity"].unique()

array(['Paddling', 'Standing', 'Surfing', 'Free Diving', 'Kite Surfing',
       'Swimming', 'Fishing', 'Walking', 'Feeding Sharks',
       'Boogie Boarding', 'Scuba Diving', 'Paddle-Skiing',
       'Body Boarding', 'Windsurfing', 'Stand-Up Paddleboarding',
       'Wading', 'Kayak Fishing For Sharks', 'Cleaning Fish',
       'Snorkeling', 'Spearfishing', nan, 'Diving', 'Fishing / Wading',
       '2 Boats Capsized', 'Night Bathing', 'Kayaking / Fishing',
       'Kayaking', 'Body Surfing', 'Swimming, Poaching Abalone',
       'Canoeing', 'Sup', 'Skimboarding', 'Touching A Shark',
       'Attempting To Lasso A Shark', 'Photo Shoot', 'Paddle Boarding',
       'Kakaying', 'Washing Hands', 'Grabbing Shark For A Selfie',
       'Kayak Fishing', 'Tagging Sharks', 'Surf Skiing', 'Surf Fishing',
       'Floating', 'Cage Diving', 'Surfng', 'Sup Foil Boarding',
       'Lobstering', 'Fishing In Alabama Deep Fishing Rodeo',
       'Fishing For Squid', 'Fishing For Sharks', 'Floating In Tube',
       

In [27]:
shark_attack.Activity.value_counts().head(30)

Surfing                           952
Swimming                          456
Spearfishing                      221
Fishing                           178
Wading                            114
Snorkeling                         85
Scuba Diving                       82
Diving                             72
Body Boarding                      64
Standing                           56
Boogie Boarding                    45
Kayaking                           35
Body Surfing                       29
Windsurfing                        19
Kayak Fishing                      14
Free Diving                        14
Walking                            13
Surf Skiing                        13
Sea Disaster                       12
Kite Surfing                       11
Surf-Skiing                        11
Freedom Swimming                    9
Sitting On Surfboard                9
Paddle Boarding                     9
Shark Fishing                       8
Playing                             8
Wade Fishing

In [28]:
shark_attack.Activity.isnull().sum()

225

In [29]:
shark_attack = shark_attack.dropna(subset=['Activity'])

In [30]:
shark_attack.shape

(3267, 7)

In [31]:
# Separar as Atividades em categorias mais importantes

# Surfing
category = lambda x: 'Surfing' if 'Surfing' in x or 'Windsurfing' in x or 'Surf' in x or 'Body Boarding' in x or 'Boogie Boarding' in x else x
shark_attack.Activity = shark_attack.Activity.map(category)

# Swimming
category = lambda x: 'Swimming' if 'Swimming' in x  or 'Swim' in x or 'Snorkeling' in x or 'Floating' in x else x
shark_attack.Activity = shark_attack.Activity.map(category)

# Wading
category = lambda x: 'Wading' if 'Wading' in x or 'Standing' in x else x
shark_attack.Activity = shark_attack.Activity.map(category)

# Diving
category = lambda x: 'Diving' if 'Diving' in x or 'Spearfishing' in x else x
shark_attack.Activity = shark_attack.Activity.map(category)

# Fishing
category = lambda x: 'Fishing' if 'Fishing' in x else x
shark_attack.Activity = shark_attack.Activity.map(category)

# Boating
category = lambda x: 'Boating' if 'Boat' in x or 'Paddling' in x or 'Paddle' in x or 'Kayaking' in x else x
shark_attack.Activity = shark_attack.Activity.map(category)


shark_attack.Activity.value_counts().head(20)

C:\Users\gisiq\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Surfing           1268
Swimming           630
Diving             517
Fishing            284
Wading             189
Boating             75
Walking             13
Sea Disaster        12
Playing              8
Bathing              7
Canoeing             7
Rowing               6
Treading Water       5
Body-Boarding        4
Freediving           4
Kite Boarding        4
Skindiving           4
Murder               4
Jumping              4
Feeding Sharks       4
Name: Activity, dtype: int64

## Coluna Fatal

In [32]:
shark_attack["Fatal"].unique()

array(['N', 'Y', nan, 'M', '2017', 'UNKNOWN'], dtype=object)

In [33]:
shark_attack.Fatal.isnull().sum()

216

In [34]:
shark_attack.Fatal = shark_attack.Fatal.fillna('No Data')

C:\Users\gisiq\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [35]:
shark_attack['Fatal']=shark_attack['Fatal'].replace({'UNKNOWN':'No Data', 'Unknown':'No Data', 'M':'N', '2017':'No Data'})
shark_attack['Fatal'].unique()

<ipython-input-35-658fbc050b5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attack['Fatal']=shark_attack['Fatal'].replace({'UNKNOWN':'No Data', 'Unknown':'No Data', 'M':'N', '2017':'No Data'})


array(['N', 'Y', 'No Data'], dtype=object)

In [36]:
shark_attack['Fatal'].value_counts()

N          2677
Y           359
No Data     231
Name: Fatal, dtype: int64

## Coluna Injury

In [37]:
shark_attack.Injury.isnull().sum()

9

In [38]:
shark_attack.Injury = shark_attack.Injury.fillna('No Data')

C:\Users\gisiq\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [39]:
shark_attack["Injury"].unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'Minor injury to left thigh',
       'Injury to left lower leg from surfboard skeg', ...,
       'No injury, speargun bitten',
       'Recovered, said to involve a 1m nurse shark', 'Left leg injured'],
      dtype=object)

In [40]:
shark_attack['Injury']=shark_attack['Injury'].replace({'No details':'No Data'})

<ipython-input-40-1c6c9f711299>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attack['Injury']=shark_attack['Injury'].replace({'No details':'No Data'})


In [41]:
# Boat, Boating and Boatomg to 1 category
category = lambda x: 'No injury' if 'No injury' in x or 'No Injury' in x else x
shark_attack.Injury = shark_attack.Injury.map(category)

category = lambda x: 'Injury' if 'laceration' in x or 'lacerations' in x or 'severed' in x or 'bitten' in x or 'Lacerations' in x or 'lacerated' in x or 'Survived' in x or 'Laceration' in x or 'wounds' in x or 'Minor' in x or 'injured' in x else x
shark_attack.Injury = shark_attack.Injury.map(category)

category = lambda x: 'Fatal' if 'Fatal' in x or 'FATAL' in x or 'fatal' in x else x
shark_attack.Injury = shark_attack.Injury.map(category)


shark_attack.Injury.value_counts().head(20)

C:\Users\gisiq\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Injury                                                                                                               2058
No injury                                                                                                             463
Fatal                                                                                                                 283
No Data                                                                                                                20
Injury to hand                                                                                                          4
Shark bites were post-mortem                                                                                            3
Abrasions                                                                                                               3
Shark involvement prior to death unconfirmed                                                                            3
Cuts on right foot      

## Coluna Country

In [42]:
shark_attack.Country.unique()

array(['Usa', 'Australia', 'Mexico', 'Brazil', 'England', 'South Africa',
       'Thailand', 'Costa Rica', 'Maldives', 'Bahamas', 'New Caledonia',
       'Ecuador', 'Malaysia', 'Libya', nan, 'Cuba', 'Mauritius',
       'New Zealand', 'Spain', 'Samoa', 'Solomon Islands', 'Japan',
       'Egypt', 'St Helena, British Overseas Territory', 'Comoros',
       'Reunion', 'French Polynesia', 'United Kingdom',
       'United Arab Emirates', 'Philippines', 'Indonesia', 'China',
       'Columbia', 'Fiji', 'Dominican Republic', 'Cayman Islands',
       'Aruba', 'Mozambique', 'Puerto Rico', 'Italy', 'Atlantic Ocean',
       'St. Martin', 'France', 'Papua New Guinea', 'Trinidad & Tobago',
       'Kiribati', 'Israel', 'Diego Garcia', 'Taiwan',
       'Palestinian Territories', 'Seychelles', 'Belize', 'Jamaica',
       'Nigeria', 'Tonga', 'Scotland', 'Canada', 'Croatia',
       'Saudi Arabia', 'Chile', 'Antigua', 'Kenya', 'Russia',
       'Turks & Caicos', 'United Arab Emirates (Uae)', 'Azores',
      

In [43]:
shark_attack.Country.value_counts().head(20)

Usa                 1553
Australia            535
South Africa         343
Brazil                95
Bahamas               81
New Zealand           58
Reunion               53
Mexico                41
New Caledonia         38
Mozambique            25
Egypt                 23
Spain                 21
Italy                 19
French Polynesia      19
Hong Kong             18
Fiji                  18
Philippines           17
Papua New Guinea      14
Tonga                 13
Japan                 13
Name: Country, dtype: int64

In [44]:
shark_attack.Country.isnull().sum()

6

In [45]:
shark_attack.Country = shark_attack.Country.fillna('No Data')

C:\Users\gisiq\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## Coluna Area

In [46]:
shark_attack.Area.unique()

array(['California', 'Georgia', 'Hawaii', 'New South Wales', 'Colima',
       'Pernambuco', 'Florida', 'Queensland', 'South Carolina',
       'Cornwall', 'Westerm Australia', 'Eastern Cape Province',
       'Hua Hin', 'Cocos Island', 'Western Australia',
       'Alifu Alifu Atoll', 'Western Cape Province', 'New Providence',
       'Alagoas', nan, 'Victoria', 'Kwazulu-Natal',
       'Boi Island, Victoria', 'Galapagos Islands', 'Fernando De Noronha',
       'Sepang', 'Holquin Province', 'Pamplemousses', 'South Australia',
       'North Island', 'New York', 'Canary Islands', 'Upolu Island',
       'Shizuoka Prefecture', 'Texas', 'Castellón', 'Massachusetts',
       'Red Sea Protectorate', 'New Providence District',
       '40 Miles Off Grand Bahama Island', 'Ascension Island', 'Majorca',
       'Washington', 'Tabasco', 'Anjouan', 'Ibiza Island', 'Marquesas',
       'South Devon', 'Sharjah,', 'Baja California Sur', 'Saint-Leu',
       'South Island', 'Luzon Island', 'Great Exuma', 'Saint-A

In [47]:
shark_attack['Area'].value_counts().head(30)

Florida                  827
Hawaii                   207
California               203
New South Wales          190
Western Cape Province    130
Queensland               115
Eastern Cape Province    114
Western Australia        110
South Carolina            93
Kwazulu-Natal             90
North Carolina            76
Pernambuco                67
South Australia           52
Texas                     44
Victoria                  38
Oregon                    29
North Island              24
Abaco Islands             21
South Island              21
Tasmania                  17
South Province            11
Alabama                   10
Grand Bahama Island       10
Virginia                  10
Quintana Roo              10
Rio De Janeiro            10
North Province             9
Gaza                       9
Guerrero                   9
Massachusetts              8
Name: Area, dtype: int64

In [48]:
shark_attack.Area.isnull().sum()

127

In [49]:
shark_attack.Area = shark_attack.Area.fillna('No Data')

C:\Users\gisiq\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## Coluna Type
Types of attack

- PROVOKED
Provoked attacks occur when a human touches, hooks, nets, or otherwise aggravates the animal. Incidents that occur outside of a shark's natural habitat, such as aquariums and research holding-pens, are considered provoked, as are all incidents involving captured sharks. Sometimes humans inadvertently provoke an attack, such as when a surfer accidentally hits a shark with a surf board.

- UNPROVOKED
Hit-and-run attack
Sneak Attack
Bump-and-bite attack

Font: https://en.wikipedia.org/wiki/Shark_attack#Types_of_attacks


In [50]:
shark_attack.Area.isnull().sum()

0

In [51]:
shark_attack.Type.value_counts()

Unprovoked      2581
Provoked         247
Invalid          230
Boating           90
Boat              61
Sea Disaster      53
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [52]:
shark_attack.Type = shark_attack.Type.replace({'Boating' : 'Boat', 'Boatomg': 'Boat', 'Invalid':'No Data', 'Questionable':'No Data'})
shark_attack.Type.value_counts()

C:\Users\gisiq\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Unprovoked      2581
Provoked         247
No Data          232
Boat             152
Sea Disaster      53
Name: Type, dtype: int64

# Exportando csv limpo

In [53]:
shark_attack.to_csv("../../shark_project/shark_attack/attacks_clean.csv", encoding='latin1', index = False)